In [1]:
                                                                                                                                                                                        # This Python 3 environment comes with many helpful analytics libraries installed
#import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib.pyplot as plt


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

****

****DATA CLeaning****

In [ ]:
#load csv files
#ddf=pd.read_csv('/kaggle/input/mvsa-multiple-attcsv/mvsa_multiple_attr.csv')
#df=pd.read_csv('/kaggle/input/simpson-attrbutes/simpson_attr_info.csv')
#df=pd.read_csv('/gdrive/MyDrive/sumana_nuig/mvsa_multiple/msva-multiple_jointlabel.csv')
df=pd.read_csv('/kaggle/input/mvsa-multiple-obj-att-clean/mvsa_multiple_attr_clean.csv')
#df1=pd.read_csv('/gdrive/MyDrive/sumana_nuig/mvsa/mvsa_attr_info1.csv')
df=df.dropna()

#df1=df1.iloc[:100:,]
df.shape
#df[:2]
#df.joint_sentiment.value_counts()

In [ ]:
#get batch of 64 we need 2844 sample
#df=df.iloc[:4288:,]

In [ ]:
import re
df["text"] = df["text"].apply(lambda x: re.sub("[\[{','}\]]", ' ', str(x)).strip())#captin text
df["class_att1"] = df["class_att"].apply(lambda x: re.sub("[\[{','}\]]", ' ', str(x)).strip())#class-attributes pair
df["class_names"] = df["object_names"].apply(lambda x: re.sub("[\[{','}\]]", ' ', str(x)).strip())#class names
df["cbt_class_attribute"]=df["text"].str.cat(df["class_att1"])#Combined caption and class-attributes pairs
df["cbt_class"]=df["text"].str.cat(df["class_names"])#Combined caption and class-attributes pairs

In [ ]:
df[:2]

**Balancing data in the dataframe**

In [ ]:
GLOVE_EMB = '/kaggle/input/glove6b300d/glove.6B.300d.txt'
EMBEDDING_DIM = 300

In [ ]:
#Training and Test
def load_tarin_test_dataset(df,col,str_len):
    column=col
    max_len=str_len
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts( df[column])
    #tokenizer.fit_on_texts(train_df.object_attr1_text)
    #tokenizer.fit_on_texts(train_df.text)
    #tokenizer1.fit_on_texts(test_data)

    word_index = tokenizer.word_index
    vocab_size = len(tokenizer.word_index) + 1
    #print("Vocabulary Size :", vocab_size)

    #jx_train = pad_sequences(tokenizer.texts_to_sequences(train_df.text), maxlen = max_len)
    x_df = pad_sequences(tokenizer.texts_to_sequences( df[column]), maxlen = max_len)
    #x_val = pad_sequences(tokenizer.texts_to_sequences(val_df.class_att), maxlen = 32)
    #jx_test = pad_sequences(tokenizer.texts_to_sequences(test_df.text), maxlen =max_len )
    #x_test = pad_sequences(tokenizer.texts_to_sequences( test[column]), maxlen =max_len )
    #y_train, y_test = train_test_split(y, test_size=1-TRAIN_SIZE,random_state=7) # Splits Dataset into Training and Testing set 
    #y_train, y_test = train_test_split(df['POSITIVE'], test_size=1-TRAIN_SIZE,random_state=7)
    embeddings_index = {}

    f = open(GLOVE_EMB, encoding="utf8")
    for line in f :
        values = line.split()
        word = value = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' %len(embeddings_index))
    embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                          EMBEDDING_DIM,
                                          weights=[embedding_matrix],
                                          input_length=max_len,#MAX_SEQUENCE_LENGTH,
                                          trainable=False)
    return x_df,vocab_size,embedding_layer

In [ ]:
!pip install keras_preprocessing

In [ ]:
#Glob tokenizer
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, Conv2D,MaxPooling2D,Bidirectional, LSTM, Dense, Input, Flatten,Dropout,GlobalMaxPool1D,MaxPooling1D,Concatenate
from tensorflow.keras.layers import SpatialDropout1D

In [ ]:
#module for extracting text features
def cnn_model_for_text(num_filters,kernel_size,tx_df,embeding_layer):
    
    num_filters=num_filters
    kernel_size=kernel_size
           
    #jsequence_input = Input(shape=(maxlen,), dtype='int32')
    #sequence_input = Input(shape=(max_len,), dtype='int32')
    #embedding_sequences = embedding_layer(sequence_input)
    embedding_sequences = embedding_layer(tx_df)
    x = SpatialDropout1D(0.2)(embedding_sequences)
    conv_layers = []
    for num_filters,kernel_size in zip(num_filters,kernel_size):
        conv_layer = Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu')(x)#(jembedding_sequences)
        pool_layer = MaxPooling1D(pool_size=max_len - kernel_size + 1)(conv_layer)
        #pool_layer = MaxPooling1D()(conv_layer)
        #pool_layer = tf.keras.layers.Reshape((1, -1))(pool_layer)
        conv_layers.append(pool_layer)

    #Concatenate all convolutional layers
    concatenate_layer = tf.keras.layers.Concatenate()(conv_layers)
    flatten_layer = Flatten()(concatenate_layer)
    x = Dropout(0.5)(flatten_layer)#(x)
    #txt = Dense(256, activation='tanh')(x)
    txt = Dense(256, activation='relu')(x)
    
    return txt

In [ ]:

#class attributes features
max_len=max([len(x.split()) for x in df.class_att1])
#tx_train,tx_test,vocab_size,embedding_layer=load_tarin_test_dataset(train_df,test_df,'text',max([len(x.split()) for x in train_df.text]))
classatt_df,vocab_size,embedding_layer=load_tarin_test_dataset(df,'class_att1',max_len)
print("Vocabulary Size :", vocab_size)
print("Training X Shape:",classatt_df.shape)
#print("Testing X Shape:",jx_test.shape)
#call cnn
kernel_sizes = [2,3,4]
num_filters = [8,16,32]
class_attributes_features=cnn_model_for_text(num_filters,kernel_sizes,classatt_df,embedding_layer)
print((np.array(class_attributes_features)).shape)


In [ ]:
def bilstm_model(tx_df,embeding_layer):
    
    embedding_sequences = embedding_layer(tx_df)
    #embedding_sequences = embedding_layer(sequence_input)
    txt = SpatialDropout1D(0.1)(embedding_sequences)
    #txt=Conv1D(100, 4,activation='relu')(txt)#(tembedding_sequences)#(x)
    txt = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2))(txt)#(tembedding_sequences)
    #ttxtb = Dense(256, activation='tanh')(txtb)
    txt = Dense(256, activation='relu')(txt)
    return txt

In [ ]:
#combined-text-features with class-attributes collection
max_len=max([len(x.split()) for x in df.cbt_class_attribute])
#tx_train,tx_test,vocab_size,embedding_layer=load_tarin_test_dataset(train_df,test_df,'text',max([len(x.split()) for x in train_df.text]))
combined_text_att,vocab_size,embedding_layer=load_tarin_test_dataset(df,'cbt_class_attribute',max_len)
print("Vocabulary Size :", vocab_size)
print("Training X Shape:",combined_text_att.shape)
#print("Testing X Shape:",jx_test.shape)
#call bilstm

combined_text_att_features=bilstm_model(combined_text_att,embedding_layer)
print((np.array(combined_text_att_features)).shape)

In [ ]:
#caption text features collection
max_len=max([len(x.split()) for x in df.text])
#tx_train,tx_test,vocab_size,embedding_layer=load_tarin_test_dataset(train_df,test_df,'text',max([len(x.split()) for x in train_df.text]))
tx_df,vocab_size,embedding_layer=load_tarin_test_dataset(df,'text',max_len)
print("Vocabulary Size :", vocab_size)
print("Training X Shape:",tx_df.shape)
#print("Testing X Shape:",tx_test.shape)

#call bilstm model
kernel_sizes = [2,3,4]
num_filters = [8,16,32]
cnn_text_features=cnn_model_for_text(num_filters,kernel_sizes,tx_df,embedding_layer)
print((np.array(cnn_text_features)).shape)

bilstm_text_features=bilstm_model(tx_df,embedding_layer)
print((np.array(bilstm_text_features)).shape)

In [ ]:
#combined-text-features with class-attributes collection
max_len=max([len(x.split()) for x in df.cbt_class_attribute])
#tx_train,tx_test,vocab_size,embedding_layer=load_tarin_test_dataset(train_df,test_df,'text',max([len(x.split()) for x in train_df.text]))
combined_text_att,vocab_size,embedding_layer=load_tarin_test_dataset(df,'cbt_class_attribute',max_len)
print("Vocabulary Size :", vocab_size)
print("Training X Shape:",combined_text_att.shape)
#print("Testing X Shape:",jx_test.shape)
#call cnn
kernel_sizes = [3,4,5]
num_filters = [16,32,64]
cbt_class_attributes_features=bilstm_model(combined_text_att,embedding_layer)
print((np.array(cbt_class_attributes_features)).shape)

**Visual Features Extraction**

In [ ]:
from tensorflow.keras.applications import VGG16,VGG19,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from keras.layers import Input,Dense, Dropout, Flatten,BatchNormalization,concatenate,Bidirectional,LSTM,GlobalAveragePooling2D
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt

**Object visual featutes**

In [ ]:
"""
np.savez('/gdrive/MyDrive/sumana_nuig/mvsa/mvsa_single_allfeatures.npz',
            img_file=np.array(img_file),
            global_features=np.array(global_features),
            local_object_features=np.array(local_object_features),
            text_features=np.array(text_features),
            class_features=np.array(class_features),
            class_attributes_features=np.array(class_attributes_features),
            cbt_class_features=np.array(cbt_class_features),
            cbt_class_attributes_features=np.array(cbt_class_attributes_features),
            target_labels=np.array(target_labels))
"""

In [ ]:
#data = np.load('/gdrive/MyDrive/sumana_nuig/mvsa/mvsa_single_allfeatures.npz')
data = np.load('/kaggle/input/mvsa-multiple-allfeatures/mvsa_multiple_allfeatures.npz')

# Access the variables inside the npz file
img_file = data['img_file']
global_features = data['global_features']
local_object_features = data['local_object_features']
text_features=data['text_features']
class_features=data['class_features']
class_attributes_features=data['class_attributes_features']
cbt_class_features=data['cbt_class_features']
cbt_class_attributes_features=data['cbt_class_attributes_features']
target_labels=data['target_labels']
print(global_features.shape,local_object_features.shape,target_labels.shape)

In [ ]:
print(train_global_features.shape, train_objects_features.shape,local_object_features.shape)

In [ ]:
"""
from sklearn.model_selection import train_test_split

# Split global and object features
train_global_features,test_global_features, train_objects_features, test_objects_features= train_test_split(global_features,
                                                                                                            local_object_features,
                                                                                                            test_size=0.1)
# split class,class_attributes features
train_text_features,test_text_features,train_class_features,test_class_features, train_class_attributes_features, test_class_attributes_features = train_test_split(
                                                                                                                          text_features,
                                                                                                                          class_features,
                                                                                                                          class_attributes_features,
                                                                                                                          test_size=0.1)
# split cbt_class_text,cbt_class_attributes_text
train_cbt_class_features,test_cbt_class_features,  train_cbt_class_attributes_features,test_cbt_class_attributes_features = train_test_split(
                                                                                                                          class_features,
                                                                                                                          class_attributes_features,
                                                                                                                          test_size=0.1)
# split target labels
y_train,y_test = train_test_split(target_labels,test_size=0.1)
"""

                                                                                                                          

In [ ]:
#Process train data
#Cross attention of object_features and class_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#text=np.array(class_features)#class features
text=np.array(class_attributes_features) #class-attributes pair features
print(text.shape)
image=np.array(local_object_features)
#image=image_features_arr
#image=global_features
print(image.shape)
# Convert numpy arrays to torch tensors
text_tensor = torch.from_numpy(text)
text_tensor=text_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
image_tensor = torch.from_numpy(image)
image_tensor=image_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class CrossAttention(nn.Module):
    def __init__(self, text_dim, image_dim):
        super(CrossAttention, self).__init__()
        self.text_dim = text_dim
        self.image_dim = image_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, text_tensor, image_tensor):
        # Compute attention weights
        
        self.text_tensor = text_tensor
        self.image_tensor = image_tensor

        # Compute attended features
    
        attention_score = torch.bmm(image_tensor.transpose(1, 2), text_tensor)
        
        attention_score = torch.softmax(attention_score, dim=1)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(image_tensor,attention_score.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(image_tensor.shape)
        # Concatenate attended features
        attended_features = torch.cat([context, image_tensor], dim=1)

        return attended_features

# Instantiate the cross-attention model
cross_att = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
attended_features1 = cross_att(text_tensor.float(), image_tensor.float())
attended_features = attended_features1.squeeze(2)
print(attended_features.shape)
# Convert the attended features back to numpy array
object_class_features = attended_features.detach().numpy()
print(object_class_features.shape)



In [ ]:

#Cross attention of global features and text_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#text=np.array(class_features)#class features
#text=np.array(class_attributes_features) #class-attributes pair features
text=np.array(text_features) #class-attributes pair features
print(text.shape)
#image=np.array(text_features)
image=np.array(global_features)
#image=image_features_arr
#image=global_features
print(image.shape)
# Convert numpy arrays to torch tensors
text_tensor = torch.from_numpy(text)
text_tensor=text_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
image_tensor = torch.from_numpy(image)
image_tensor=image_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class CrossAttention(nn.Module):
    def __init__(self, text_dim, image_dim):
        super(CrossAttention, self).__init__()
        self.text_dim = text_dim
        self.image_dim = image_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, text_tensor, image_tensor):
        # Compute attention weights
        
        self.text_tensor = text_tensor
        self.image_tensor = image_tensor

        # Compute attended features
    
        attention_score = torch.bmm(image_tensor.transpose(1, 2), text_tensor)
        
        attention_score = torch.softmax(attention_score, dim=1)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(image_tensor,attention_score.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(image_tensor.shape)
        # Concatenate attended features
        attended_features = torch.cat([context, image_tensor], dim=1)

        return attended_features

# Instantiate the cross-attention model
cross_att = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
#attended_features2 = cross_att(text_tensor.float(), image_tensor.float())
#attended_features2 = attended_features2.squeeze(2)
#print(attended_features2.shape)
#cross attention with global image and text
global_text_features1 = cross_att(text_tensor.float(), image_tensor.float())
global_text_features = global_text_features1.squeeze(2)
print(global_text_features.shape)
# Convert the attended features back to numpy array
global_text_features = global_text_features.detach().numpy()
print(global_text_features.shape)



In [ ]:
#Additive attention of caption text_features and class_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#class_text=np.array(class_features)
class_text=np.array(class_attributes_features)
#print(text.shape)
caption_text=np.array(text_features)
#print(image.shape)
# Convert numpy arrays to torch tensors
class_tensor = torch.from_numpy(class_text)
class_tensor=class_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
caption_tensor = torch.from_numpy(caption_text)
caption_tensor=caption_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class AdditiveAttention(nn.Module):
    def __init__(self, class_dim, caption_dim):
        super(AdditiveAttention, self).__init__()
        self.class_dim = class_dim
        self.caption_dim = caption_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, class_tensor, caption_tensor):
        # Compute attention weights
        
        self.class_tensor = class_tensor
        self.caption_tensor = caption_tensor

        # Compute attended features
        attention_score = class_tensor+caption_tensor
        tanh_score = torch.tanh(attention_score)
        print(tanh_score.shape)
        attention_score = torch.softmax(tanh_score, dim=1)
        print(attention_score.shape)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(attention_score,class_tensor.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(caption_tensor.shape)
        #print(caption_tensor.transpose(1,2).shape)
        # Concatenate attended features
        #attended_features = torch.cat([context, caption_tensor.transpose(1,2)], dim=1)
        attended_features = torch.cat([context[:, :, 0], caption_tensor[:, :, 0]], dim=1)

        #attended_features = torch.cat([context, caption_tensor], dim=0)

        return attended_features

# Instantiate the cross-attention model
additive_att = AdditiveAttention(class_dim=class_text.shape[1], caption_dim=caption_text.shape[1])
#cross_att = CAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
additive_features_text_class = additive_att(class_tensor.float(), caption_tensor.float())
additive_features_text_class1 = additive_features_text_class.unsqueeze(2)
#print(additive_attention2.shape)
# Convert the attended features back to numpy array
#additive_features_text_class1 = additive_features_text_class1.detach().numpy()
print(additive_features_text_class1.shape)

In [ ]:

#2nd level cross attention of global_text_features1 and additive_features_text_class1
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#text=np.array(class_features)#class features
text=np.array(class_attributes_features) #class-attributes pair features
print(text.shape)
image=np.array(text_features)
#image=image_features_arr
#image=global_features
print(image.shape)
# Convert numpy arrays to torch tensors
text_tensor = torch.from_numpy(text)
text_tensor=text_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
image_tensor = torch.from_numpy(image)
image_tensor=image_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class CrossAttention(nn.Module):
    def __init__(self, text_dim, image_dim):
        super(CrossAttention, self).__init__()
        self.text_dim = text_dim
        self.image_dim = image_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, text_tensor, image_tensor):
        # Compute attention weights
        
        self.text_tensor = text_tensor
        self.image_tensor = image_tensor

        # Compute attended features
    
        attention_score = torch.bmm(image_tensor.transpose(1, 2), text_tensor)
        
        attention_score = torch.softmax(attention_score, dim=1)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(image_tensor,attention_score.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(image_tensor.shape)
        # Concatenate attended features
        attended_features = torch.cat([context, image_tensor], dim=1)

        return attended_features

# Instantiate the cross-attention model
cross_att = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
attended_features2 = cross_att(text_tensor.float(), image_tensor.float())
#attended_features2 = attended_features2.squeeze(2)
print(attended_features2.shape)
# Convert the attended features back to numpy array
#object_class_features = attended_features.detach().numpy()
#print(object_class_features.shape)

In [ ]:
#reconstruct the image after attention
# Load the pre-trained VAE model
#vae_model = tf.keras.models.load_model('vae_model.h5')
# Reconstruct the image from the joint representation
#reconstructed_img = vae_model.predict(cross_attention)
#import matplotlib.pyplot as plt

# Rescale the pixel values from -1 to 1 to 0 to 255
#reconstructed_img = (reconstructed_img[0] + 1) / 2 * 255

# Convert the reconstructed image to integers
#reconstructed_img = reconstructed_img.astype('uint8')

# Display the reconstructed image
#plt.imshow(reconstructed_img)
#plt.show()

In [ ]:
concatenated_text_global_image=np.concatenate([attended_features3,additive_attention2_features],axis=1)
print(concatenated_text_global_image.shape)

In [ ]:
input_shape = (concatenated_text_global_image.shape[1])
print(input_shape)

In [ ]:
#concatenated_text_global_image=np.concatenate([attended_features3,additive_attention2_features],axis=1)
def bilstmodel(additive_features_text_class):
    input_shape = (np.expand_dims(additive_features_text_class,axis=2))
    #reshaped_tensor = input_shape.permute(1, 0, 2)
    #input_shape=(additive_attention2.shape,)
    #embedding_sequences = embedding_layer(tx_df)
    #embedding_sequences = embedding_layer(sequence_input)
    #txt = SpatialDropout1D(0.1)(input_shape)
    #txt=Conv1D(100, 4,activation='relu')(txt)#(tembedding_sequences)#(x)
    txt = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2))(input_shape)#(tembedding_sequences)
    #ttxtb = Dense(256, activation='tanh')(txtb)
    txt = Dense(1024, activation='relu')(txt)
    return txt
bilstm_additive_attention2_features=bilstmodel(numphy_object_image_text_pair)
print((np.array(bilstm_additive_attention2_features)).shape)
bilstm_additive_attention2_features=np.expand_dims(bilstm_additive_attention2_features,axis=2)
print(bilstm_additive_attention2_features.shape)
  

In [ ]:
#2nd level cross attention of global_text_features1 and additive_features_text_class1
image=global_text_features1
#text=additive_attention2
#text=torch.tensor(bilstm_additive_attention2_features)
#text=attended_features2
text=additive_features_text_class1
#text=additive_attention1
cross_att2 = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
image_tensor=image
text_tensor=text
global_image_text_pair = cross_att2(text_tensor.float(), image_tensor.float())
#attended_features = attended_features.squeeze(2)
print(global_image_text_pair.shape)
reshape_global_image_text_pair=crossattention_level2[:, :, 0]
# Convert the attended features back to numpy array
numphy_global_image_text_pair = reshape_global_image_text_pair.detach().numpy()
print(numphy_global_image_text_pair.shape)

In [ ]:
print(attended_features1.shape,additive_features_text_class1.shape)

In [ ]:
#  2nd cross-attention between cros_attention(attention 1) and additive attention(attention 2) model
image=attended_features1
#text=additive_attention2
#text=torch.tensor(bilstm_additive_attention2_features)
#text=attended_features2
text=additive_features_text_class1
#text=additive_attention1
cross_att2 = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
image_tensor=image
text_tensor=text
object_image_text_pair = cross_att2(text_tensor.float(), image_tensor.float())
#attended_features = attended_features.squeeze(2)
print(object_image_text_pair.shape)
reshape_object_image_text_pair=crossattention_level2[:, :, 0]
# Convert the attended features back to numpy array
numphy_object_image_text_pair = reshape_object_image_text_pair.detach().numpy()
print(cross_attention_features_level2.shape)


In [ ]:
#  cross-attention between cros_attention(attention 1) and cross attention(global image+text) model
image=global_image_text_pair
#text=additive_attention2
text=torch.tensor(bilstm_additive_attention2_features)
#text=attended_features2
#text=additive_features1
#text=additive_attention1
cross_att2 = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
image_tensor=image
text_tensor=text
crossattention_image_text_pair = cross_att2(text_tensor.float(), image_tensor.float())
#attended_features = attended_features.squeeze(2)
print(crossattention_image_text_pair.shape)
crossattention_image_text_pairs=crossattention_image_text_pair[:, :, 0]
# Convert the attended features back to numpy array
crossattention_image_text_pairs = crossattention_image_text_pairs.detach().numpy()
print(crossattention_image_text_pairs.shape)

In [ ]:
"""
#Train multimodal features creation
#Cross attention of object_features and class_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#text=np.array(class_features)#class features
text=np.array(train_class_attributes_features) #class-attributes pair features
print(text.shape)
image=np.array(train_objects_features)
#image=image_features_arr
#image=global_features
print(image.shape)
# Convert numpy arrays to torch tensors
text_tensor = torch.from_numpy(text)
text_tensor=text_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
image_tensor = torch.from_numpy(image)
image_tensor=image_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class CrossAttention(nn.Module):
    def __init__(self, text_dim, image_dim):
        super(CrossAttention, self).__init__()
        self.text_dim = text_dim
        self.image_dim = image_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, text_tensor, image_tensor):
        # Compute attention weights
        
        self.text_tensor = text_tensor
        self.image_tensor = image_tensor

        # Compute attended features
    
        attention_score = torch.bmm(image_tensor.transpose(1, 2), text_tensor)
        
        attention_score = torch.softmax(attention_score, dim=1)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(image_tensor,attention_score.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(image_tensor.shape)
        # Concatenate attended features
        attended_features = torch.cat([context, image_tensor], dim=1)

        return attended_features

# Instantiate the cross-attention model
cross_att = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
attended_features1 = cross_att(text_tensor.float(), image_tensor.float())
attended_features = attended_features1.squeeze(2)
print(attended_features.shape)
# Convert the attended features back to numpy array
object_class_features = attended_features.detach().numpy()
print(object_class_features.shape)

#Additive attention of caption text_features and class_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#class_text=np.array(class_features)
class_text=np.array(train_class_attributes_features)
#print(text.shape)
caption_text=np.array(train_text_features)
#print(image.shape)
# Convert numpy arrays to torch tensors
class_tensor = torch.from_numpy(text)
class_tensor=class_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
caption_tensor = torch.from_numpy(caption_text)
caption_tensor=caption_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class AdditiveAttention(nn.Module):
    def __init__(self, class_dim, caption_dim):
        super(AdditiveAttention, self).__init__()
        self.class_dim = class_dim
        self.caption_dim = caption_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, class_tensor, caption_tensor):
        # Compute attention weights
        
        self.class_tensor = class_tensor
        self.caption_tensor = caption_tensor

        # Compute attended features
        attention_score = class_tensor+caption_tensor
        tanh_score = torch.tanh(attention_score)
        print(tanh_score.shape)
        attention_score = torch.softmax(tanh_score, dim=1)
        print(attention_score.shape)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(attention_score,class_tensor.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(caption_tensor.shape)
        #print(caption_tensor.transpose(1,2).shape)
        # Concatenate attended features
        #attended_features = torch.cat([context, caption_tensor.transpose(1,2)], dim=1)
        attended_features = torch.cat([context[:, :, 0], caption_tensor[:, :, 0]], dim=1)

        #attended_features = torch.cat([context, caption_tensor], dim=0)

        return attended_features

# Instantiate the cross-attention model
additive_att = AdditiveAttention(class_dim=class_text.shape[1], caption_dim=caption_text.shape[1])
#cross_att = CAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
additive_features = additive_att(class_tensor.float(), caption_tensor.float())
additive_attention2 = additive_features.unsqueeze(2)
#print(additive_attention2.shape)
# Convert the attended features back to numpy array
text_class_features = additive_features.detach().numpy()
print(text_class_features.shape)

# Instantiate the 2nd cross-attention between cros_attention(attention 1) and additive attention(attention 2) model
image=attended_features1
text=additive_attention2
cross_att2 = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
image_tensor=image
text_tensor=text
crossattention_level2 = cross_att2(text_tensor.float(), image_tensor.float())
#attended_features = attended_features.squeeze(2)
print(crossattention_level2.shape)
cross_attention_features_level2=crossattention_level2[:, :, 0]
# Convert the attended features back to numpy array
train_cross_attention_features_level2 = cross_attention_features_level2.detach().numpy()
print(train_cross_attention_features_level2.shape)
"""


In [ ]:
"""
#Test multimodal features creation
#Cross attention of object_features and class_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#text=np.array(class_features)#class features
text=np.array(test_class_attributes_features) #class-attributes pair features
print(text.shape)
image=np.array(test_objects_features)
#image=image_features_arr
#image=global_features
print(image.shape)
# Convert numpy arrays to torch tensors
text_tensor = torch.from_numpy(text)
text_tensor=text_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
image_tensor = torch.from_numpy(image)
image_tensor=image_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class CrossAttention(nn.Module):
    def __init__(self, text_dim, image_dim):
        super(CrossAttention, self).__init__()
        self.text_dim = text_dim
        self.image_dim = image_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, text_tensor, image_tensor):
        # Compute attention weights
        
        self.text_tensor = text_tensor
        self.image_tensor = image_tensor

        # Compute attended features
    
        attention_score = torch.bmm(image_tensor.transpose(1, 2), text_tensor)
        
        attention_score = torch.softmax(attention_score, dim=1)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(image_tensor,attention_score.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(image_tensor.shape)
        # Concatenate attended features
        attended_features = torch.cat([context, image_tensor], dim=1)

        return attended_features

# Instantiate the cross-attention model
cross_att = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
attended_features1 = cross_att(text_tensor.float(), image_tensor.float())
attended_features = attended_features1.squeeze(2)
print(attended_features.shape)
# Convert the attended features back to numpy array
object_class_features = attended_features.detach().numpy()
print(object_class_features.shape)

#Additive attention of caption text_features and class_features with batch size
import torch
import torch.nn as nn
import numpy as np
#train_classat_features, rtrain_object_features, rtrain_object_features)
#class_text=np.array(class_features)
class_text=np.array(test_class_attributes_features)
#print(text.shape)
caption_text=np.array(test_text_features)
#print(image.shape)
# Convert numpy arrays to torch tensors
class_tensor = torch.from_numpy(text)
class_tensor=class_tensor.unsqueeze(2)
#text_tensor=text_tensor.permute(0, 1)
caption_tensor = torch.from_numpy(caption_text)
caption_tensor=caption_tensor.unsqueeze(2)
#image_tensor=image_tensor.permute(0, 1)

# Define a cross-attention model
class AdditiveAttention(nn.Module):
    def __init__(self, class_dim, caption_dim):
        super(AdditiveAttention, self).__init__()
        self.class_dim = class_dim
        self.caption_dim = caption_dim
        #self.linear_text = nn.Linear(text_dim, image_dim)
        #self.linear_image = nn.Linear(image_dim, text_dim)

    def forward(self, class_tensor, caption_tensor):
        # Compute attention weights
        
        self.class_tensor = class_tensor
        self.caption_tensor = caption_tensor

        # Compute attended features
        attention_score = class_tensor+caption_tensor
        tanh_score = torch.tanh(attention_score)
        print(tanh_score.shape)
        attention_score = torch.softmax(tanh_score, dim=1)
        print(attention_score.shape)
        #attention_score = attention_score.unsqueeze(2) # transpose back to [batch_size, sequence_length, feature_dim]
        context = (torch.bmm(attention_score,class_tensor.transpose(1,2)))
        #image_attended = torch.matmul(text_att.transpose(0, 1), image)
        #image_attended = image_attended.transpose(0, 1)
        print(context.shape)
        print(caption_tensor.shape)
        #print(caption_tensor.transpose(1,2).shape)
        # Concatenate attended features
        #attended_features = torch.cat([context, caption_tensor.transpose(1,2)], dim=1)
        attended_features = torch.cat([context[:, :, 0], caption_tensor[:, :, 0]], dim=1)

        #attended_features = torch.cat([context, caption_tensor], dim=0)

        return attended_features

# Instantiate the cross-attention model
additive_att = AdditiveAttention(class_dim=class_text.shape[1], caption_dim=caption_text.shape[1])
#cross_att = CAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
additive_features = additive_att(class_tensor.float(), caption_tensor.float())
additive_attention2 = additive_features.unsqueeze(2)
#print(additive_attention2.shape)
# Convert the attended features back to numpy array
text_class_features = additive_features.detach().numpy()
print(text_class_features.shape)

# Instantiate the 2nd cross-attention between cros_attention(attention 1) and additive attention(attention 2) model
image=attended_features1
text=additive_attention2
cross_att2 = CrossAttention(text_dim=text.shape[1], image_dim=image.shape[1])
#cross_att = CrossAttention(text_dim=(text.shape), image_dim=(image.shape))

# Perform cross-attention
image_tensor=image
text_tensor=text
crossattention_level2 = cross_att2(text_tensor.float(), image_tensor.float())
#attended_features = attended_features.squeeze(2)
print(crossattention_level2.shape)
cross_attention_features_level2=crossattention_level2[:, :, 0]
# Convert the attended features back to numpy array
test_cross_attention_features_level2 = cross_attention_features_level2.detach().numpy()
print(test_cross_attention_features_level2.shape)
"""


In [ ]:
print(text_features.shape)

**Simple Concatenation of Image and  **

In [ ]:
#all experiment with balance sampling
#combined global(image features), capton text features
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from collections import Counter


#all experiment with balance sampling
#combined global(image features), capton text features
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn import svm
#with strategy.scope(): #when TPU is used
#with tf.device('/device:GPU:0'): #when GPU is used
ros = RandomOverSampler()
#combined_features=np.concatenate([object_class_features,text_features,global_features], axis=1)
#combined_features = np.concatenate([cross_attention_features_level2, cbt_class_attributes_features,attended_features3], axis=1)
#combined_features = np.concatenate([cross_attention_features_level2, cbt_class_attributes_features,attended_features3,global_features,text_features], axis=1)
#combined_features = np.concatenate([crossattention_image_text_pairs, global_features,cbt_class_attributes_features], axis=1)
combined_features = np.concatenate([text_features,class_attributes_features,global_features,local_object_features],axis=1)
#combined_features_ros, target_labels_ros = ros.fit_resample(combined_features, target_labels)
#X_train, X_test, y_train, y_test = train_test_split(combined_features_ros, target_labels_ros, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(combined_features, target_labels, test_size=0.1)
#X_train, X_test, y_train, y_test = train_test_split(combined_features, df.image_label, test_size=0.1)
#X_train, y_train = ros.fit_resample(X_train, y_train) 
#X_test, y_test = ros.fit_resample(X_train, y_train) 
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape) 

In [ ]:
from sklearn import svm
with tf.device('/device:GPU:0'):
# create SVM classifier
  clf = svm.SVC(kernel='linear', C=1, probability=True)

  # train the classifier on the training data
  clf.fit(X_train, y_train)

  # evaluate the classifier on the test data
  #print(accuracy)
  # evaluate the classifier on the test data
  accuracy = clf.score(X_test, y_test)
  predicted = clf.predict(X_test)

  #ypredicted = np.argmax(predicted,axis=1)
  #y_predicted = np.where(predicted>0.5,1,0)
  print('Results of SVM')
  from sklearn.metrics import classification_report
  print(classification_report(y_test,predicted,digits=4))

In [ ]:
## get dummies values for labels
y_train=pd.get_dummies(y_train)
def get_label_code(text):
    if text=='positive':
        return 2
    elif text=='neutral':
        return 1
    else :
        return 0
my_vectorized_function = np.vectorize(get_label_code)
y_test = my_vectorized_function(y_test)
#y_test=y_test.apply(get_label_code)
X_train = np.expand_dims(X_train, axis=-1)

In [ ]:
from keras.models import Sequential
# assume X_train, y_train, X_test, y_test are NumPy arrays containing the training and test data
input_shape = (combined_features.shape[1], 1) # assuming the data is a 1D sequence of length 1056

# define the CNN1D model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=4, activation='relu', input_shape=input_shape))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
#cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
#cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(3, activation='softmax'))
#cnn_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',
             metrics=['categorical_accuracy'])
#####
#X_train = np.expand_dims(X_train, axis=-1)
cnn_history=cnn_model.fit(x=X_train,validation_split=0.1,y= y_train,batch_size=128, epochs=10)
#com_history=fusion_modelfit(x=X_train, y_train, epochs=10,batch_size=128, validation_split=0.1,validation_data=(X_test, y_test))

#accuracy vs loss function
s, (at, al) = plt.subplots(2,1)
at.plot(cnn_history.history['categorical_accuracy'], c= 'b')
at.plot(cnn_history.history['val_categorical_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['Fusion_Res_cnn_train', 'Fusion_Res_cnn_val'], loc='upper left')

al.plot(cnn_history.history['loss'], c='m')
al.plot(cnn_history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')
cpredicted = cnn_model.predict(X_test)
#com_score=fusion_model.evaluate(y_test)
#print('Test Loss:', com_score[0])
#print('Test accuracy:', com_score[1])
cypredicted = np.argmax(cpredicted,axis=1)
#y_predicted = np.where(predicted>0.5,1,0)
print('Results of CNN model')
from sklearn.metrics import classification_report
print(classification_report(y_test,cypredicted,digits=4))

In [ ]:
from keras.models import Sequential
# assume X_train, y_train, X_test, y_test are NumPy arrays containing the training and test data
input_shape = (combined_features.shape[1], 1) # assuming the data is a 1D sequence of length 1056

# define the CNN1D model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=4, activation='relu', input_shape=input_shape))
cnn_model.add(MaxPooling1D(pool_size=2))
#cnn_model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
#cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
#cnn_model.add(Dense(128, activation='relu'))
#cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(3, activation='softmax'))
#cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',
            metrics=['categorical_accuracy'])
#####
#X_train = np.expand_dims(X_train, axis=-1)
cnn_history=cnn_model.fit(x=X_train,validation_split=0.1,y= y_train,batch_size=128, epochs=20)
#com_history=fusion_modelfit(x=X_train, y_train, epochs=10,batch_size=128, validation_split=0.1,validation_data=(X_test, y_test))

#accuracy vs loss function
s, (at, al) = plt.subplots(2,1)
at.plot(cnn_history.history['categorical_accuracy'], c= 'b')
at.plot(cnn_history.history['val_categorical_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['Fusion_Res_cnn_train', 'Fusion_Res_cnn_val'], loc='upper left')

al.plot(cnn_history.history['loss'], c='m')
al.plot(cnn_history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')
cpredicted = cnn_model.predict(X_test)
#com_score=fusion_model.evaluate(y_test)
#print('Test Loss:', com_score[0])
#print('Test accuracy:', com_score[1])
cypredicted = np.argmax(cpredicted,axis=1)
#y_predicted = np.where(predicted>0.5,1,0)
print('Results of CNN model')
from sklearn.metrics import classification_report
print(classification_report(y_test,cypredicted,digits=4))

In [ ]:
# define the BiLSTM model
from keras.models import Sequential
# assume X_train, y_train, X_test, y_test are NumPy arrays containing the training and test data
input_shape = (combined_features.shape[1], 1)
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(64,dropout=0.5,recurrent_dropout=0.5), input_shape=input_shape))
#lstm_model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
#lstm_model.add(Bidirectional(LSTM(64)))
lstm_model.add(Dense(3, activation='softmax'))
#lstm_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['categorical_accuracy'])
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy',
            metrics=['categorical_accuracy'])
  
with tf.device('/device:GPU:0'): #when GPU is used
  #Excecute Bilstm
  lstm_history=lstm_model.fit(x=X_train, y=y_train,validation_split=0.1,batch_size=128,  epochs=5)
  #com_history=fusion_modelfit(x=X_train, y_train, epochs=10,batch_size=128, validation_split=0.1,validation_data=(X_test, y_test))

  #accuracy vs loss function
  s, (at, al) = plt.subplots(2,1)
  at.plot(lstm_history.history['categorical_accuracy'], c= 'b')
  at.plot(lstm_history.history['val_categorical_accuracy'], c='r')
  at.set_title('model accuracy')
  at.set_ylabel('accuracy')
  at.set_xlabel('epoch')
  at.legend(['Fusion_bilstm_train', 'Fusion_bilstm_val'], loc='upper left')

  al.plot(lstm_history.history['loss'], c='m')
  al.plot(lstm_history.history['val_loss'], c='c')
  al.set_title('model loss')
  al.set_ylabel('loss')
  al.set_xlabel('epoch')
  al.legend(['train', 'val'], loc = 'upper left')
  cpredicted = lstm_model.predict(X_test)
  #com_score=fusion_model.evaluate(y_test)
  #print('Test Loss:', com_score[0])
  #print('Test accuracy:', com_score[1])
  cypredicted = np.argmax(cpredicted,axis=1)
  #y_predicted = np.where(predicted>0.5,1,0)
  print('Results of Bilstm model')
  from sklearn.metrics import classification_report
  print(classification_report(y_test,cypredicted))

In [ ]:
"""

# Set the initial learning rate, decay steps, decay rate, and weight decay
initial_learning_rate = 0.001
decay_steps = 10
decay_rate = 0.1
weight_decay = 1e-3

# Create the learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate
)

# Create the optimizer with weight decay
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    decay=weight_decay
)

# Compile the model
#model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
#model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
"""
